In [4]:
import pandas as pd
import os
import datetime as dt
import numpy as np
import pyodbc
from preston_fx import *

In [5]:
# dir = os.getcwd()
# os.chdir("S:\\ISR\\Branch - Investment Risk\\Risk Oversight\\Working\\Oleg\\Working\\Dashboard Code")

In [6]:
data_dir = "data"
map_dir = "mapping"
report_date = dt.date(2022, 9, 30) # define what date you want conversion rate

# report file names

expo_file = "Exposure - September 30, 2022_v4.xlsx"
efront_file = "efront Data in RiskRaw.xlsx"
BCI_file = "Portfolio All Assets - BCI_Base20221016-191307.xlsx"
leverage_file = "I&RR Leverage December 31, 2021_vF_Sent20220615.xlsx"
WARF_file = "Dashboard_WARF_2022_Sep.xlsx"
report_file = "2022 Jul 1 - Sep 30 I&RR Reporting.xlsx"
fx_rate_file = "fx rate.xlsx"

# mapping file names

name1_file = "Portfolio All Asset Name Key.xlsx"
name2_file = "IRR Leverage File Key.xlsx"
name3_file = "efront file 1 name key.xlsx"
map_file = "sector-mapping-exposure.xlsx" 
fund_name_cash_file = "cash flow fund name.xlsx" 

# Uses Full investment names as key to find Investment Type, region, MV CAD, and Sector
GP_map_file ="GP Mapping table.xlsx"
fund_name_file = "reporting fund name.xlsx"

In [7]:
name1_df =  pd.read_excel(
     io = f"{map_dir}\\{name1_file}", header = 0)
bci_dict = name1_df.set_index('Investee Name').to_dict()['Actual Investment Name']

name2_df =  pd.read_excel(
     io = f"{map_dir}\\{name2_file}", header = 0)
leverage_dict = name2_df.set_index('Investee Name').to_dict()['Actual Investment Name']

name3_df =  pd.read_excel(
     io = f"{map_dir}\\{name3_file}", header = 0)
efront_dict = name3_df.set_index('Investee Name').to_dict()['Actual Investment Name']

fund_name_df =  pd.read_excel(
     io = f"{map_dir}\\{fund_name_file}", header = 0)
fund_dict = fund_name_df.set_index('Investee Name').to_dict()['Actual Investment Name']

fund_name_cash_df =  pd.read_excel(
     io = f"{map_dir}\\{fund_name_cash_file}", header = 0)
cash_fund_dict = fund_name_df.set_index('Investee Name').to_dict()['Actual Investment Name'] 

map_df = pd.read_excel(
    io = f"{map_dir}\\{map_file}", header = 0,
    sheet_name = "Sheet1")

map_df.set_index('Asset Name', inplace = True)

map_dict = map_df.to_dict()

type_dict = map_dict["Investment Type"]
region_dict = map_dict["Region"]
sector_dict = map_dict["Sector"]

GP_map_df = pd.read_excel(
    io = f"{map_dir}\\{GP_map_file}", 
    sheet_name = "GP Mapping",
    usecols = "A,B")

GP_map_df.columns = ['Asset Name', 'GP']        
GP_map_df.set_index("Asset Name", inplace = True)

In [8]:
# retrieve from SQl server the 

# excel connection
# fx_rate_df = pd.read_excel(
#     io = f"{data_dir}\\{fx_rate_file}",
#     sheet_name = "Inputs", parse_dates = True)

# fx_rate_df = fx_rate_df[fx_rate_df["EFFECTIVE_DATE"] == "2022-09-30"]

fx = preston_fx_rate()
fx = fx[fx['EFFECTIVE_DATE'] == report_date]

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [ ]:
expo_df = pd.read_excel(
    io = f"{data_dir}\\{expo_file}", 
    sheet_name = "Market Values",
    skiprows = range(3), #Remember to test 2 or 3
    usecols = "B,I")

expo_df.columns = ['Asset Name', 'Market Values (C$ Hedged)']        
expo_df.set_index("Asset Name", inplace = True)
drop_list = list(expo_df[expo_df.index.str.contains('Total')].index) + ['IFT']
expo_df.drop(axis=0, index = drop_list, inplace = True) # drop subtotals

In [ ]:
# DF - Efront 1

efront_df1 = pd.read_excel(
    io = f"{data_dir}\\{efront_file}", 
    sheet_name = "Company overview",
    skiprows = 0,
    usecols ="C,D,N",
    parse_dates = True
)

efront_df1["First_Investment_Date"] = pd.to_datetime(efront_df1["First_Investment_Date"]).dt.normalize()

short_names1 = efront_df1["Company_Name"].to_list()

full_names1 = []
for short in short_names1:
    if short in efront_dict:
        full_names1.append(efront_dict.get(short))
    else:
        full_names1.append(short)
        
efront_df1.columns = ['Company_Name', 'First Investment Date', 'Strategy']        
efront_df1["Asset Name"] = full_names1
efront_df1.drop("Company_Name", axis = 1, inplace = True)
efront_df1.set_index("Asset Name", inplace = True)

In [ ]:
# DF - Levearge
leverage_df = pd.read_excel(
    io = f"{data_dir}\\{leverage_file}", 
    sheet_name = "2021 Direct",
    skiprows = range(3),
    usecols ="A,C,D")

short_names_leverage = leverage_df["Investment"].to_list()

full_names_leverage = []
for short in short_names_leverage:
    if short in leverage_dict:
        full_names_leverage.append(leverage_dict.get(short))
    else:
        full_names_leverage.append(short)

leverage_df.columns = ['Investment', 'EV Dec 31, 2021 (Local)', 'Net Debt (Local)']     
leverage_df["Asset Name"] = full_names_leverage
leverage_df.drop("Investment", axis = 1, inplace = True)
leverage_df.set_index("Asset Name", inplace = True)

In [ ]:
BCI_df = pd.read_excel(
    io = f"{data_dir}\\{BCI_file}", 
    sheet_name = "Portfolio Summary (Millions",
    skiprows = range(1),
    usecols ="C,N,Q,R,AE")

short_names_bci = BCI_df["Investee Name"].to_list()

full_names_bci = []
for short in short_names_bci:
    if short in bci_dict:
        full_names_bci.append(bci_dict.get(short))
    else:
        full_names_bci.append(short)
BCI_df["Asset Name"] = full_names_bci
BCI_df.drop("Investee Name", axis = 1, inplace = True)
BCI_df = BCI_df[:-1]
BCI_df.set_index("Asset Name", inplace = True)

In [ ]:
report_df1 = pd.read_excel(
    io = f"{data_dir}\\{report_file}", 
    sheet_name = "Unfunded Commitments Funds",
    skiprows = 1,
    usecols ="B,F,H,J,L")

short_names_fund = report_df1["Fund"].to_list()

full_names_fund = []
for short in short_names_fund:
    if short in fund_dict:
        full_names_fund.append(fund_dict.get(short))
    else:
        full_names_fund.append(short)
        

report_df1.columns = ['Asset Name', 
                    'Commitment CAD', 
                    'Remaining Commitment CAD', 
                    'Total Called CAD', 
                    'Total Distributed CAD']
report_df1["Asset Name"] = full_names_fund
report_df1.set_index("Asset Name", inplace = True)

In [ ]:
report_df2 = pd.read_excel(
    io = f"{data_dir}\\{report_file}", 
    sheet_name = "Unfunded Commitments Directs",
    skiprows = 1,
    usecols ="B,F,H")

report_df2.columns = ['Asset Name', 'Commitment CAD', 'Remaining Commitment CAD']
report_df2.set_index("Asset Name", inplace = True)

In [9]:
report_df3 = pd.read_excel(
    io = f"{data_dir}\\{report_file}", 
    sheet_name = "Cash Flow Directs",
    skiprows = 1,
    usecols ="B,E,I")

short_names_report = report_df3["Company"].to_list()

full_names_report = []
for short in short_names_report:
    if short in efront_dict:
        full_names_report.append(efront_dict.get(short))
    else:
        full_names_report.append(short)
        

report_df3.columns = ['Asset Name', 'Investments', 'Proceeds CAD']
report_df3["Asset Name"] = full_names_report
# report_df3.drop("Company", axis = 1, inplace = True)
report_df3.set_index("Asset Name", inplace = True)

# short_names1 = efront_df1["Company_Name"].to_list()

# full_names1 = []
# for short in short_names1:
#     if short in efront_dict:
#         full_names1.append(efront_dict.get(short))
#     else:
#         full_names1.append(short)
        
# efront_df1.columns = ['Company_Name', 'First Investment Date', 'Strategy']        
# efront_df1["Asset Name"] = full_names1
# efront_df1.drop("Company_Name", axis = 1, inplace = True)
# efront_df1.set_index("Asset Name", inplace = True)

In [10]:
report_df4 = pd.read_excel(
    io = f"{data_dir}\\{report_file}", 
    sheet_name = "Cash Flow Funds",
    skiprows = 1,
    usecols ="C,F,H")

short_names_fund2 = report_df4["Fund"].to_list()

full_names_fund2 = []
for short in short_names_fund2:
    if short in cash_fund_dict :
        full_names_fund2.append(cash_fund_dict.get(short))
    else:
        full_names_fund2.append(short)
        

report_df4.columns = ['Asset Name', 'Cash Called CAD', 'Cash Distributed CAD']
report_df4["Asset Name"] = full_names_fund2
report_df4.set_index("Asset Name", inplace = True)

report_df4 = report_df4.groupby('Asset Name').sum()

In [12]:
# CREDIT RATINGS 
rt_direct_df = pd.read_excel(
    io = f"{data_dir}\\{WARF_file}", 
    sheet_name = "Overview",
    skiprows = range(37),
    usecols ="B,C,D",
    nrows = 33
)

rt_direct_df.set_index("Asset Name", inplace = True)
# djfhdjkghdkjgjkdhgdjk

rt_dir_sect_df = pd.read_excel(
    io = f"{data_dir}\\{WARF_file}", 
    sheet_name = "Overview",
    skiprows = range(75),
    usecols ="B,C",
    nrows = 9
)

rt_fund_sect_df = pd.read_excel(
    io = f"{data_dir}\\{WARF_file}", 
    sheet_name = "Overview",
    skiprows = range(88),
    usecols ="B,C",
    nrows = 4
)

In [13]:
rt_direct_df

,Credit Rating,Note
Asset Name,,
Ag Partners Capital LLC,BBB-,"Used to be proxy, now internal"
Aguas Andinas S.A,A-,"Used to be proxy, now external"
Arteris,BB,Internal
Caddo sustainable Timberlands,BB,Internal
Cleco Partners LP,BBB-,Internal
Connaught Oil & Gas,BBB-,Proxy: Weighed average of proxy distribution
Corex Resources,BBB-,Proxy: Weighed average of proxy distribution
Corix Infrastructure Inc. (regulated),BBB-,Internal
Corix Infrastructure Inc. (unregulated),BB,Internal


In [ ]:
# Step 1 - create a master_df with only the index consisted of the asset names
asset_names = list(set([i for i in list(set(expo_df.index))]))

master_df = pd.DataFrame({"Investment Name": asset_names})
master_df.set_index("Investment Name", inplace = True)

In [ ]:
report_df1 = master_df.join(report_df1, how="left")
report_df2 = master_df.join(report_df2, how="left")
report_df1 = report_df1.fillna(report_df2)

In [ ]:
# Step 2 - merge DataFrames in dfs into the master df one by one using a loop. Joint type is "outer"
# for df in dfs:
#     master_df = master_df.join(df, how = "outer") # try changing joint type
dfs = [expo_df, efront_df1, BCI_df, leverage_df, report_df3, rt_direct_df, GP_map_df, map_df, report_df1, report_df4]

master_df = master_df.join(dfs, how = "left")
master_df.index.name = "Investment Name"

# master_df = master_df.groupby("Investment Name").sum()

In [ ]:
# cl_x = 'Commitment CAD_x'
# cl2_x = 'Remaining Commitment CAD_x'
# cl_y = 'Commitment CAD_y'
# cl2_y = 'Remaining Commitment CAD_y'

# master_df['Commitment CAD'] = master_df[[cl_x, cl_y]].sum(axis=1, min_count=1)
# master_df['Remaining Commitment CAD'] = master_df[[cl2_x, cl2_y]].sum(axis=1, min_count=1)
# master_df.drop(labels=[cl_x, cl_y, cl2_x, cl2_y], axis = 1, inplace = True)

In [ ]:
static_col = ['Investment Type', 'Market Values (C$ Hedged)', 'First Investment Date', 'Strategy',
       'Credit Rating', 'Note', 'GP', 'Region', 'Sector']
dynamic_col = ['Investment Type', 'Market Values (C$ Hedged)'] + [i for i in master_df.columns if i not in static_col]

master_static = master_df[static_col]
master_dynamic = master_df[dynamic_col]

In [ ]:
# Step 3 - export master_df into a spreadsheet
master_static.to_excel("master_static_df.xlsx")
master_dynamic.to_excel("master_dynamic_df.xlsx")

PermissionError: [Errno 13] Permission denied: 'master_dynamic_df.xlsx'